In [1]:
!pip -q install -U transformers accelerate peft bitsandbytes sentencepiece sacrebleu evaluate huggingface_hub datasets fsspec
import torch, random, os, json, itertools, textwrap
from datasets import load_dataset, DatasetDict
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import evaluate, tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.8/514.8 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
BASE_MODEL = "meta-llama/Llama-3.2-1B"
SRC_LANG, TGT_LANG = "English", "Spanish"
# LoRA hyper-params
lora_conf = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
)
BATCH = 4
EPOCHS = 2
MAX_LEN = 256


In [6]:
bnb_conf = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True,
                              bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token


model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL, device_map="auto", quantization_config=bnb_conf)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_conf)
model.print_trainable_parameters()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

trainable params: 11,272,192 || all params: 1,247,086,592 || trainable%: 0.9039


In [7]:
def format_prompt(text, src=SRC_LANG, tgt=TGT_LANG):
    return (f"Translate the following text from {src} to {tgt}:\n"
            f"{src}: {text}\n"
            f"{tgt}:")

In [8]:
from transformers import DefaultDataCollator

# ---------- tiny slice just for demo ----------
subset = load_dataset("opus_books", "en-es", split="train[:3%]")
temp   = subset.train_test_split(test_size=0.20, seed=42)
train_raw, test_raw = temp["train"], temp["test"]
tmp    = train_raw.train_test_split(test_size=0.20, seed=42)
train_raw, valid_raw = tmp["train"], tmp["test"]

ds = DatasetDict(train=train_raw, validation=valid_raw, test=test_raw)

def preprocess(ex):
    src, tgt = ex["translation"]["en"], ex["translation"]["es"]

    # 1️⃣ build full prompt **plus** answer in one sequence
    full_text = format_prompt(src) + " " + tgt + tokenizer.eos_token
    enc = tokenizer(full_text, truncation=True, max_length=MAX_LEN)

    # 2️⃣ mask the prompt part with -100 so loss is only on the answer
    prompt_ids = tokenizer(format_prompt(src), add_special_tokens=False)["input_ids"]
    labels = [-100] * len(prompt_ids) + enc["input_ids"][len(prompt_ids):]

    assert len(labels) == len(enc["input_ids"]), "label/ids length mismatch"

    enc["labels"] = labels
    enc["reference"] = tgt          # keep plain text for BLEU
    return enc

tokenized = ds.map(preprocess, remove_columns=ds["train"].column_names, num_proc=4)
tokenized.set_format(type="torch",
                     columns=["input_ids", "labels", "attention_mask", "reference"])

README.md:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/93470 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1794 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/449 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/561 [00:00<?, ? examples/s]

In [9]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True, return_tensors="pt")

args = TrainingArguments(
    output_dir="./lora-llama32-translate",
    per_device_train_batch_size=BATCH,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=50,
    num_train_epochs=EPOCHS,
    lr_scheduler_type="linear",
    bf16=True,
    save_total_limit=2,
    report_to="none"
)

trainer = Trainer(model=model,
                  args=args,
                  train_dataset=tokenized["train"],
                  eval_dataset=tokenized["validation"],
                  data_collator=data_collator)

trainer.train()
output_path = "/content/drive/MyDrive/lora-llama32-en-es"
model.save_pretrained(output_path)
tokenizer.save_pretrained(output_path)

No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentr

Step,Training Loss
50,1.359100
100,1.245200
150,1.084700
200,0.968600


('/content/drive/MyDrive/lora-llama32-en-es/tokenizer_config.json',
 '/content/drive/MyDrive/lora-llama32-en-es/special_tokens_map.json',
 '/content/drive/MyDrive/lora-llama32-en-es/tokenizer.json')

In [10]:
from peft import PeftModel, PeftConfig
# Load base model + tokenizer
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL, device_map="auto", torch_dtype=torch.float16
)
base_tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
base_tokenizer.pad_token = base_tokenizer.eos_token
base_tokenizer.padding_side = "left"

# Load LoRA-adapted model (same base + adapter weights)
tuned_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL, device_map="auto", torch_dtype=torch.float16
)
tuned_model = PeftModel.from_pretrained(tuned_model, "/content/drive/MyDrive/lora-llama32-en-es")

tuned_tokenizer = base_tokenizer  # same tokenizer

In [11]:
def translate(model, tokenizer, prompt, max_new_tokens=64):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)
    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=False
    )
    return tokenizer.decode(out[0], skip_special_tokens=True).strip()

In [12]:
def format_prompt(text, src="English", target="Spanish"):
    return f"Translate this from {src} to {target}:\n{src}: {text}\n{target}:"

samples = [
    "I need to get braces for my overbite.",
    "How long does the treatment usually take?",
    "My dentist said I have a crossbite.",
    "Will it hurt when I get my aligners?",
    "I lost my last tray. What should I do?"
]

for s in samples:
    prompt = format_prompt(s)
    print(f"\n📌 Input: {s}")
    print("🔹 Base Model:", translate(base_model, base_tokenizer, prompt))
    print("🔸 Tuned Model:", translate(tuned_model, tuned_tokenizer, prompt))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



📌 Input: I need to get braces for my overbite.


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🔹 Base Model: Translate this from English to Spanish:
English: I need to get braces for my overbite.
Spanish: Necesito que me hagan unas ortodoncia para mi exagerada boca.
I need to get braces for my overbite.
I need to get braces for my overbite.
I need to get braces for my overbite.
I need to get braces for my overbite.
I need to get braces for


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

🔹 Base Model: Translate this from English to Spanish:
English: Will it hurt when I get my aligners?
Spanish: ¿Sarará doloroso cuando me ponga mis ortodoncias?
The answer is yes, but it will be a lot less painful than you think. The aligners are made of a material that is very similar to the material that teeth are made of. The aligners are made of a material that is


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🔸 Tuned Model: Translate this from English to Spanish:
English: Will it hurt when I get my aligners?
Spanish: ¿Le molestarán los alargadores?

📌 Input: I lost my last tray. What should I do?


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🔹 Base Model: Translate this from English to Spanish:
English: I lost my last tray. What should I do?
Spanish: ¿Qué debo hacer ahora que he perdido mi última bandeja?
I lost my last tray. What should I do?
I lost my last tray. What should I do?
I lost my last tray. What should I do?
I lost my last tray. What should I do?
I lost my last tray
🔸 Tuned Model: Translate this from English to Spanish:
English: I lost my last tray. What should I do?
Spanish: Perdí mi última bandeja. ¿Qué hago?
